# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

In [4]:
SCROLL_PAUSE_TIME = 1

# 以下是用Selenium模擬下拉網頁動作，讓網頁更新

last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")  # 滾動到 the bottom of the page

while True:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    print('網頁更新中...')
    time.sleep(SCROLL_PAUSE_TIME)
    
    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        print('到達頁面底端')
        break

    last_height = new_height

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [5]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [6]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 雙美去年EPS 1.84元　每股配發現金股利1.5元

▲左起為雙美副總經理林廣恒、董事長林齊國、總經理林賴美枝、中國執行董事林泉源。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導
雙美生技（4728）今（27）日公告2019年合併營收達5.94億元，年增95.4％；稅後淨利1億元，年增34.54％；稅後每股盈餘(EPS)為1.84元，營收、獲利刷新歷史新高，董事會決議配發每股現金股利1.5元，以今日平盤價58.5元計算，現金殖利率達2.56％。
雙美表示，去年致力於擴大中國大陸市場版圖，將雙美膠原蛋白植入劑I 號、III號陸續通過認證並核准上市，爭取膠原蛋白產品的專業醫學美容注射技術與更多醫生及機構的合作機會，帶動旗下膠原蛋白植入劑銷售表現暢旺，創造中國大陸地區營收年增率倍數成長，推升產能利用率滿載。
雙美指出，2019年整體毛利率86.54％，較去年增加7.51個百分點；營業利益率29.46％，較去年增加8.81個百分點，惟一次性於去年第4季認列建廠重新規劃、調整代理商通路等相關費用與損失，共2525萬元，是第4季單季營運表現呈虧損的主要原因。
雙美表示，董事會決議通過2019年股利政策，考量公司財務體質健全且資本公積充沛，為將營運成果回饋全體股東，2019年擬配發每股現金股利1.5元，現金配發率達81.52％，以今日平盤價58.5元計算，現金殖利率達2.56％，以期將整體營運成果與全體股東共享。
展望今年第1季，雙美指出，雖因新冠肺炎疫情擴散，使得出貨的工作天數較少與消費者外出消費力道減緩，但觀察目前中國大陸疫情在全民貫徹防疫工作下而逐漸穩定控制，醫美診所也在2月17日後陸續復工開業，且雙美大陸子公司於客戶復工後即接獲多張膠原蛋白植入劑訂單出貨。
雙美表示，未來雙美將以膠原蛋白原料為基礎，研發面膜、保養品等護膚品，開拓線上線下通路，另一方面，將膠原蛋白植入劑原料升級進而拉開適應症範圍，透過擁有中國藥監局(NMPA)械字號認證雙美I號之MC32膠原美塑療程，搶攻美塑療法市場的份額，增添整體營運表現穩健的成長動能。
雙美強調，疫情期間仍持續推進Sunmax雙美膠原蛋白植入劑的品牌力與市場份額，積極透過線上直播、Face

▲美股道瓊上漲 1351.62  點。回頭看台指期今（27）日開盤上漲268 點，來到 9899 點。（圖／路透）
記者陳心怡／台北報導
美國參議院通過規模 2 兆美元的刺激法案，美股週四（26 日）四大指數收漲皆逾 5%，道瓊回歸牛市 (道瓊已自近期低點回漲約 21%)，站上 22000 大關。回頭看台指期今（27）日開盤上漲268 點，來到 9899 點。
美股道瓊指數上漲 1351.62  點，或 6.38%，收 22,552.17 點。標普 500 指數上漲 154.51 點，或 6.24%，收 2,630.07 點。那斯達克指數上漲 413.24 點，或 5.60%，收 7,797.54 點。費城半導體指數上漲 100.04 點，或 6.79%，收 1,572.61 點。
嘉信理財負責交易和衍生品的副總裁弗雷德里克（Randy Frederick）表示，看到投資人在前一天大漲後繼續買進令人鼓舞，因為大家已經一個月沒有看到這種情況了。不過這並不能保證股市已經觸底，這是一個築底過程開始的跡象。
截至今早8：47，台指期上漲 244 點，來到 9875 點。
----------------------------------------------------------------------

[11] 殯葬大亨吃「泰山」！澳門黑幫變大老闆　拉攏詹家砸6億搶經營權

▲朱國榮聯合泰山當權派以外的詹家成員，讓泰山再爆經營權之爭。（圖／鏡週刊提供，下同）
圖、文／鏡週刊
 
週三（25日）龍邦一則公告引發各界注意，因為該公司董事會決議，預計再砸6億元買進3.1萬張泰山股票，事實上，過去一年來龍邦已多次買進泰山，截至目前為止，持股約25.94％，若加上未來預計買進的股票，持股比例將達到32.23％，也讓經營權之爭的問題浮上檯面。
本刊調查，龍邦大量買進泰山正是為了取得經營權，為此龍邦幕後老闆、國寶集團總裁朱國榮還拉攏當權派以外的詹家成員，包括前董事長詹岳霖、前副董事長詹晉嘉，目標取得過半數股權。
據了解，目前詹岳霖一派約有10％股權，以詹晉嘉為首的一派持有5％，加計龍邦手上已有的持股以及未來欲買進的股權，三方勢力約有47％股權，知情人士透露：「未來龍邦也不排除找持股超過10％的保力達合作！」就是要取得50％以上，藉此拿下控制權。
除了在市場買股、找盟友，龍邦近期

▲ 中央銀行。（圖／記者李毓康攝）
記者李瑞瑾／台北報導
央行於今年3月19日理事會決議，各項政策利率自3月20日起，分別調降0.25個百分點，重貼現率、擔保放款融通利率、短期融通利率分別調降為1.125%、1.5%、3.375%，改寫歷史新低紀錄，而大多數銀行已配合調整其牌告利率。
央行指出，為反映銀行存款利率之調降，自3月27日起，金融機構存放央行準備金乙戶（應提準備額之55 %）存款之利率，源自活期性存款部分，調整為年息0.068%（原為0.146%）；源自定期性存款部分，調整為年息0.56%（原為0.82%）。
另外，源自外資之新臺幣活期存款，仍依央行99年12月30日函之規定，全部不給付利息。
----------------------------------------------------------------------

[23] 台股賣壓力竭　慣性改變

美元指數轉弱，新台幣匯率轉強，美股有機會築底，外資開始回補台積電、鴻海等權值股，台股可望持續反彈輪動擴底，等待基本面轉好。
文．馮泉富
截至三月二十四日止的過去一周，全球主要股市雖仍動盪不已，但仔細觀察後，卻出現一絲絲不一樣的地方。
首先，現階段新冠病毒重災區在歐洲，德國法蘭克福指數、法國巴黎指數於波段跌幅達四成之後，已五個交易日未再破低；英國富時100指數也同樣五個交易日未創低；亞洲的日經指數三月二十四日的一根長紅直接吞噬前五根日Ｋ線並收復十日線；韓國首爾指數三月二十四日大漲八．六％，已三日不破低；台灣加權指數與香港恆生指數亦三日不破低。
這樣的現象顯示短線慣性開始改變，在交易結構的內涵可以解釋為，急於求現的、想賣的、沒信心的大致上都出場了，因此，賣壓減輕了。
觀察美股VIX波動率指數，也由日前的歷史高位八二．六九降至三月二十三日的六一．五九，顯示美股也有機會進入築底階段。
對台股而言，還有很重要的一個訊號，就是三月二十四日當天外資買超五十二億新台幣，終結連續十二天的賣超慣性。
再者，近期因國際上美元流動性吃緊，外資法人直接將台股當成提款機，權值股台積電遭逢ETF、退休基金、對沖基金、主權基金等來自四面八方的求現賣壓，累計今年以來外資賣超台積電近七十萬張，提款約二千一百億元，使台積電蒙受龐大的股價壓力；如果外資慣性改變，反賣為買，台股加權指數當有機會展開反彈波，投資人宜好好把握。
美元指

▲ 華南金總部。（圖／記者吳靜君攝）
記者李瑞瑾／台北報導
華南永昌證券從事衍生性商品交易未實現損失達該公司淨值3%，損失高達34億元，證期局表示，華南永昌證券目前已作出相對應避險措施，不致於讓損失再擴大。
華南永昌證券從事權證交易因避險不及損失慘重，市場關注華南永昌的虧損是否可以縮小。華南金董事長張雲鵬今（26）日指出，因為之前有期貨空單避險，近期股票上漲，該權證交易恐怕更不利。
證期局官員指出，華南永昌證券目前的避險措施，有一部份是放空期貨，另一部分則是以選擇權避險，官員並表示，華南永昌目標是將收斂放空避險部位，轉移至選擇權，不管股價上或下都能夠保住。
另外，證期局官員表示，避險操作是公司作業問題，金管會並沒有實際上去看分布多少，但已確認避險措施有做好，依目前來看，最大損失為34.1億元，不致於讓損失再擴大。
證期局官員表示，也請證交所針對權證發行量大的券商作追蹤，了解是否有出現類似的避險不及狀況，但並沒有發現到類似情況，證交所也會持續關注。
----------------------------------------------------------------------

[32] 壽險業2月賺497億元　淨值再逼近2兆續創歷史新高

▲ 壽險業2月匯損達408億元，稅前利益則仍有497億元。（圖／pixabay）
記者李瑞瑾／台北報導
金管會保險局今（26）日公布保險業匯兌損益數字，2月匯率較2019年底貶值0.7%，壽險業累計淨匯損為408億元，但稅前利益仍達497億元；壽險業2月淨值則為1兆9,963億元，續創歷史新高紀錄。
根據保險局統計，2020年2月底壽險業匯兌獲利有722億元，但避險損失有1160億元，加上動用外匯價格變動準備金30億元，因此2月壽險業累計淨匯損為408億元。
至於淨值方面，統計顯示，保險業今年2月淨值達2兆1,321億元，較上年同期成長43.8%，續創歷史新高，其中壽險業淨值來到1兆9,963億元，年增達47%，同時續創歷史新高紀錄，而產險淨值則達到1,358億元，年成長9.5%。
整體來看，2月壽險業稅前獲利497億元，較上年同期增加321億元，年成長182.4%；產險業稅前獲利29億元，較上年同期增減少1億元，年減3.3%，累計今年2月保險業稅前獲利達526億元，較上年同期增加320億元，年增155.3%。
到今年